In [ ]:
import sqlite3
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

## SQL

In [ ]:
con = sqlite3.Connection("data/hw2.sqlite")

In [ ]:
sql_restaurants = """
SELECT
	rest.id AS "rest_id",
	rt_types.type AS "restaurant_type",
	zip.lat AS "lat",
	zip.lng AS "lng"
FROM
	restaurants AS rest
LEFT JOIN restaurant_types AS rt_types ON
	rest.type = rt_types.type_id
LEFT JOIN zip_codes as zip ON
	rest.zip = zip.zip;
"""

In [ ]:
sql_tourists = """
SELECT
t.id AS tourist_id,
t.first_name AS "first_name",
t.last_name AS "last_name",
t.age AS "age",
zip.lat AS "lat",
zip.lng AS "lng"
FROM 
tourists AS t
LEFT JOIN zip_codes AS zip ON
t.zip = zip.zip;
"""

In [ ]:
restaurants = pd.read_sql(sql=sql_restaurants, con=con)
restaurants.info()

In [ ]:
tourists = pd.read_sql(sql=sql_tourists, con=con)
tourists.info()

## KNN

In [ ]:
def knn(base: np.ndarray, entities: np.ndarray, k: int) -> np.ndarray:
    distance = cdist(entities, base)
    return np.argsort(distance, axis = 1)[:, :k]

In [ ]:
knn(base = restaurants.loc[:, "lat":"lng"].to_numpy(), entities= tourists.loc[:, "lat":"lng"].to_numpy(), k = 5)

## KMeans

In [ ]:
'''
Выбираем случайные точки
Для каждой точки определяем расстояние до каждого ресторана
Определяем ближайшую точку для каждого ресторана 
Группируем рестораны по ближайшим точкам (k групп)
Для каждой группы определяем "центральную" точку = новая точка в каждой группе
Цикл проверки:
Определяем для ресторанов ближайшую точку из новых точек
Группируем рестораны по новым точкам
Если рестораны в группе поменялись - обновляем группу, опредеяем новую 
центральную точку в группе и повторяем цикл
Если не поменялись - завершаем цикл.
???
Profit!!!
'''

In [ ]:
# Шаг 1: выбор произвольных стартовых точек:
def get_random_centroids(X: pd.DataFrame, k: int) -> pd.DataFrame:
    return X.sample(n = k).to_numpy()

In [ ]:
# Шаг 2: KMeans:
def kmeans(restaurants: np.ndarray, centroids: np.ndarray) -> np.ndarray:
    k = len(centroids)
    # Для каждой точки определяем расстояние до каждого ресторана 
    distances = cdist(centroids, restaurants)
    #Определяем ближайшую точку для каждого ресторана
    nearest_centroid = np.argmin(distances, axis=0)
    # Группируем рестораны по ближайшим точкам (k групп)
    groups = [np.where(nearest_centroid == c)[0] for c in range(k)]
    # Для каждой группы определяем "центральную" точку = новая точка в каждой группе
    new_centroids = np.array([np.nanmean(restaurants[groups[i]], axis=0) for i in range(k)])
    # проверяем новые координаты
    while True:
        distances = cdist(new_centroids, restaurants, metric='euclidean')
        nearest_centroid = np.argmin(distances, axis=0)
        new_groups = [np.where(nearest_centroid == i)[0] for i in range(k)]
    # проверяем, поменялись ли группы
        if all([np.array_equal(groups[i], new_groups[i]) for i in range(k)]):
            break
        groups = new_groups
        new_centroids = np.array([np.nanmean(restaurants[groups[i]], axis=0) for i in range(k)])
    for i in range(k):
        print(f"Number of restaurants in group {i+1} is", len(new_groups[i]))
    return new_groups

In [ ]:
rand_coords = get_random_centroids(restaurants.loc[:, "lat":"lng"], 5)
rand_coords

In [ ]:
kmeans(restaurants = restaurants.loc[:, "lat":"lng"].to_numpy(), centroids = rand_coords)

#### Если выбирать рандомные сгенерированные координаты (а не из списка ресторанов), то бывают случаи, когда kmeans "падает" и закидывает все рестораны в одну группу

In [ ]:
restaurants.loc[:, "lat"].min()

In [ ]:
restaurants.loc[:, "lat"].max()

In [ ]:
restaurants.loc[:, "lng"].min()

In [ ]:
restaurants.loc[:, "lng"].max()

In [ ]:
random_lng = np.random.uniform(low = -77.0000000, high = -73.200000, size = 2000)
#random_lng

In [ ]:
random_lat = np.random.uniform(low = 38.6600000, high = 42.500000, size = 2000)
#random_lat

In [ ]:
random_coords = pd.DataFrame(np.dstack((random_lat, random_lng)).reshape(2000, 2), columns = ['lat', "lng"])
random_coords
# pandas - чтобы потом использовать pd.df.sample()

In [ ]:
rand_centroids = get_random_centroids(random_coords, 5)
rand_centroids

In [ ]:
kmeans(restaurants = restaurants.loc[:, "lat":"lng"].to_numpy(), centroids = rand_centroids)

## DBScan

In [ ]:
'''
1) Рассчитываем расстояние между ресторанами (точками) и записываем его в 
список/ таблицу (расстояние всех от всех). 

2) Выбираем случайным образом ресторан (точку) и определяем, сколько 
ресторанов есть на расстоянии "е" от неё, записываем их в список (делаем 
группу)

3) Для каждой точки в группе определяем, есть ли на расстоянии "е" от неё ещё
точки: если есть, то добавляем в группу. 
Если на расстоянии "е" точек больше нет, то "закрываем" группу, переходим 
к следующей точке и выполняем ту же проверку.

4) Если точка уже есть в какой-либо группе, переходим к следующей точке.
5) Если в группе точек меньше, чем N, то помечаем группу как "outlier".
'''

In [ ]:
# 1)
dist_table = cdist(
                    restaurants.loc[:, "lat":"lng"], 
                    restaurants.loc[:, "lat":"lng"]
                  )
dist_table[1997]#.mean()

In [ ]:
np.random.seed(1)
np.random.randint(2000)

In [ ]:
e = 0.4
np.random.seed(1)
group1 = np.where(dist_table[np.random.randint(2000)] <= e)[0]
group1.size

In [ ]:
# аналогично предыдущей строке
e = 0.4
np.random.seed(1)
group_1 = np.asarray(dist_table[np.random.randint(2000)] <= e).nonzero()[0]
group_1.size

In [ ]:
# только определяет точки на расстоянии e и отмечает их
def scanner(coords: np.ndarray, point: int, e: float) -> np.array:
    # таблица расстояний
    dist_table = cdist(
                    coords, 
                    coords
                  )
    initial_group = np.where(dist_table[point] <= e)[0] #группа точек на расстоянии <=e от исходной
    clustered_points = [] # все проверенные точки
    cluster_points = [] # точки текущего кластера
    for item in initial_group:
        if dist_table[0][item] not in clustered_points:
            clustered_points.append(item)
        cluster_points.append(item)
    
    return np.array(cluster_points).size, np.array(clustered_points).size

In [ ]:
np.random.seed(1)
scanner(restaurants.loc[:, "lat":"lng"], point = np.random.randint(2000), e = 0.4)

In [ ]:
class DBScanner_rest:

    def __init__(self, restaurants: np.ndarray, e: float, N: int):
        self.restaurants = restaurants
        self.e = e
        self.N = N
        self.distances = cdist(self.restaurants, self.restaurants) #расстояния между рестораноми
        self.cluster_points = np.array([]) #список точек (индексов) текущего кластера
        self.visited = [] # Список уже изученных точек
        self.groups = {} # Список групп и точек, которые в них входят
        self.outliers = {'outliers': []}
        
    # Шаг 2: выбор произвольной точки, которую мы ещё не исследовали:
    def get_random_point(self):
        point = np.random.randint(2000)
        if point not in self.visited:
            return point
        point = get_random_point()
    
    # Шаг 3: Поиск точек, которые находятся на расстоянии не более чем `e` от выбранной точки:
    def find_nearest_points(self, point): 
        indices = np.where(self.distances[point] <= self.e)[0]
        #if indices.size < 2: # только для случая, когда такую точку передали изначально
            #self.cluster_points = np.append(self.cluster_points, point)
            #self.visited.append(point)
            #self.outliers['outliers'].append(point)
        #else: 
        for idx in indices: # добавляем точки в текущий кластер
            if idx not in self.cluster_points:
                self.cluster_points = np.append(self.cluster_points, idx)
        self.visited.append(point)
            
        return self.cluster_points
        
# Шаг 4-5: Проверка других точек в группе на предмет наличия "соседей", до сих пор не вошедших в группу:        
    def full_cluster(self, init_cluster: np.ndarray) -> np.ndarray:
        for cluster_point in init_cluster:
            indices = np.where(self.distances[int(cluster_point)] <= self.e)[0]
            for idx in indices:
                if idx in self.visited:
                    continue
                else: 
                    sub_cluster = self.find_nearest_points(idx)
                    for sub_cluster_point in sub_cluster:
                        if sub_cluster_point not in self.cluster_points:
                            self.cluster_points = np.append(self.cluster_points, sub_cluster_point)
        if self.cluster_points.size > len(init_cluster):
            self.full_cluster(self.cluster_points)
        return self.cluster_points
# Шаг 6: Проход по всем точкам и группировка (повторяем шаги 1-5 до тех пор, пока не останется неисследованных точек):
    def create_groups(self):
        n = 1
        for point in range(self.distances[0].size):
            if point not in self.visited:
                self.groups[n] = self.full_cluster(list(self.find_nearest_points(point)))
                n += 1
                self.cluster_points = np.array([])
        for i in range(len(self.groups)):
            if len(self.groups[i+1]) <= self.N:
                self.outliers['outliers'].append(self.groups[i+1])
        return self.groups

In [ ]:
test = DBScanner_rest(restaurants = restaurants.loc[:, "lat":"lng"], e = 0.4, N = 5)

In [ ]:
test.create_groups()

In [ ]:
test.outliers

In [ ]:
for i in range(len(test.groups)):
    print(test.groups[i+1].size)

In [ ]:
#test.find_nearest_points(1061)

In [ ]:
#test.cluster_points

In [ ]:
#full_cluster = test.full_cluster(test.cluster_points)

In [ ]:
#full_cluster.size

### Аналог cdist

In [ ]:
def euclidean_distance(XA: np.ndarray, XB: np.ndarray) -> np.ndarray:
    return np.sqrt(np.sum((XA[:, np.newaxis, :] - XB[np.newaxis, :, :]) ** 2, axis=2))

In [ ]:
euclidean_distance(restaurants.loc[:, "lat":"lng"].to_numpy(), restaurants.loc[:, "lat":"lng"].to_numpy())